In [34]:
import yfinance as yf
import pandas_ta as ta
import joblib
import shutil

# Download data and immediately flatten columns
data = yf.download("RELIANCE.NS", period="2mo", interval="1d")
data.columns = data.columns.get_level_values(0)  # Keep only the first level
data.drop(index=data.index[-1],inplace=True)
# Now pandas_ta will work normally
data.ta.rsi(length=14, append=True)
data.ta.sma(length=20, append=True)

data.dropna(inplace=True)
print(data[['Close', 'RSI_14', 'SMA_20']].tail())

C:\Users\shres\AppData\Local\Temp\ipykernel_22428\1345289958.py:7: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download("RELIANCE.NS", period="2mo", interval="1d")
[*********************100%***********************]  1 of 1 completed

Price             Close     RSI_14       SMA_20
Date                                           
2025-07-07  1541.500000  71.519261  1473.775018
2025-07-08  1537.599976  69.756186  1478.730017
2025-07-09  1519.000000  61.916739  1482.235016
2025-07-10  1517.199951  61.199919  1486.015015
2025-07-11  1495.199951  53.107457  1489.380011


In [ ]:
data['Future_Close'] = data['Close'].shift(-5)
data['Future_Return_5d'] = ((data['Future_Close'] - data['Close']) / data['Close']) * 100

In [36]:
data['Future_Close'] = data['Close'].shift(-5)
data['Future_Return_5d'] = ((data['Future_Close'] - data['Close']) / data['Close']) * 100
data.dropna(inplace=True)
data.reset_index(drop=True,inplace=True)
print(data[['Close','Future_Close','Future_Return_5d']].tail(10))

Price        Close  Future_Close  Future_Return_5d
9      1456.800049   1500.599976          3.006585
10     1450.800049   1528.400024          5.348771
11     1467.300049   1518.800049          3.509848
12     1495.300049   1517.800049          1.504715
13     1515.400024   1527.300049          0.785273
14     1500.599976   1541.500000          2.725578
15     1528.400024   1537.599976          0.601933
16     1518.800049   1519.000000          0.013165
17     1517.800049   1517.199951         -0.039537
18     1527.300049   1495.199951         -2.101755


In [37]:
#Selecting features(X) and target(y)
features = ['RSI_14','SMA_20']
X=data[features]
y=data['Future_Return_5d']


In [38]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

In [39]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)
model = RandomForestRegressor(n_estimators=100,random_state=42)
model.fit(X_train,y_train)

preds = model.predict(X_test)
print(preds,y_test)

mae = mean_absolute_error(y_test,preds)
print("MAE:",mae)

[1.16932305 2.95234934 3.37819964 1.97860579] 0    -0.507476
5     1.369487
11    3.509848
1    -1.297539
Name: Future_Return_5d, dtype: float64
MAE: 1.6668638210289435


In [40]:
joblib.dump(model, 'pnl_model.pkl')
shutil.move('pnl_model.pkl','backend/ml/pnl_model.pkl')

'backend/ml/pnl_model.pkl'